In [1]:
import datetime
start_time="2017-04-01"
end_time="2020-04-01"
start_timestamp = datetime.datetime.strptime(start_time, '%Y-%m-%d').replace(tzinfo=datetime.timezone.utc).timestamp()
end_timestamp = datetime.datetime.strptime(end_time, '%Y-%m-%d').replace(tzinfo=datetime.timezone.utc).timestamp()

In [2]:
import requests
url = 'https://min-api.cryptocompare.com/data/v2/histohour'
params = {
    'fsym': 'BTC',
    'tsym': 'USDT',
    'e': 'binance',
    'toTs': end_timestamp,
    'limit': 2000
}
response = requests.get(url, params=params).json()
last_timestamp = response['Data']['TimeFrom']
data = response['Data']['Data']

In [3]:
import pandas as pd
df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['time'],unit='s')
df.tail()

,time,close,high,low,open,volumefrom,volumeto,conversionType,conversionSymbol,datetime
1996,1585684800,6467.50,6494.90,6447.75,6463.47,2503.51,16207519.69,force_direct,,2020-03-31 20:00:00
1997,1585688400,6442.61,6485.00,6420.00,6467.50,2112.04,13622726.48,force_direct,,2020-03-31 21:00:00
1998,1585692000,6431.55,6448.00,6420.64,6442.61,1197.65,7707041.79,force_direct,,2020-03-31 22:00:00
1999,1585695600,6410.44,6440.99,6386.89,6431.55,2279.88,14626241.96,force_direct,,2020-03-31 23:00:00
2000,1585699200,6305.12,6426.52,6239.34,6410.44,9061.28,57230360.58,force_direct,,2020-04-01 00:00:00


In [4]:
while (last_timestamp > start_timestamp):
    params = {
        'fsym': 'BTC',
        'tsym': 'USDT',
        'e': 'binance',
        'toTs': last_timestamp - 1,
        'limit': 2000
    }
    response = requests.get(url, params=params).json()
    last_timestamp = response['Data']['TimeFrom']
    data = response['Data']['Data'] + data

In [5]:
df = pd.DataFrame(data)
df = df[df.time >= start_timestamp]
df['datetime'] = pd.to_datetime(df['time'],unit='s')
df.drop(columns=['time', 'conversionType', 'conversionSymbol'], inplace=True)
df.rename(columns={'volumefrom': 'volume', 'volumeto': 'baseVolume'}, inplace=True)
df.tail()

,close,high,low,open,volume,baseVolume,datetime
28009,6467.50,6494.90,6447.75,6463.47,2503.51,16207519.69,2020-03-31 20:00:00
28010,6442.61,6485.00,6420.00,6467.50,2112.04,13622726.48,2020-03-31 21:00:00
28011,6431.55,6448.00,6420.64,6442.61,1197.65,7707041.79,2020-03-31 22:00:00
28012,6410.44,6440.99,6386.89,6431.55,2279.88,14626241.96,2020-03-31 23:00:00
28013,6305.12,6426.52,6239.34,6410.44,9061.28,57230360.58,2020-04-01 00:00:00


In [6]:
df.head()

,close,high,low,open,volume,baseVolume,datetime
1709,0.0,0.0,0.0,0.0,0.0,0.0,2017-04-01 00:00:00
1710,0.0,0.0,0.0,0.0,0.0,0.0,2017-04-01 01:00:00
1711,0.0,0.0,0.0,0.0,0.0,0.0,2017-04-01 02:00:00
1712,0.0,0.0,0.0,0.0,0.0,0.0,2017-04-01 03:00:00
1713,0.0,0.0,0.0,0.0,0.0,0.0,2017-04-01 04:00:00


In [7]:
df.to_csv('./output.csv', index=False)